In [2]:
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
include("../src/utils.jl")
include("../src/unit_commitments.jl")
include("../src/plots.jl")
include("../src/network_analysis.jl")
ENV["COLUMNS"]=120; 

gen_df, gen_var_long, loads_long, network = process_data("../WECC")

zone_dict = Dict(zip(gen_df.zone, gen_df.zone_name));
line_dict = Dict(zip(network.network_lines, network."transmission path name"));

In [ ]:
# demand multipliers, gas multipliers 
scenario_multipliers = [
    (0.90, 0.60),
    (0.95, 0.80),
    (1.00, 1.00),
    (1.05, 1.20),
    (1.10, 1.50),
    (1.10, 2.00)
]
SCENARIOS = []

for (demand_mult, gas_mult) in scenario_multipliers
    this_scenario = deepcopy(base_inputs)
    this_scenario[:demand] .*= demand_mult

    # Not important -- reverse the VarOM cost to be able to update the gas price in-situ
    thermal_G = this_scenario[:generators].Heat_rate_MMBTU_per_MWh .!= 0
    derived_Cost_per_MMBtu = (this_scenario[:generators].Var_Cost[thermal_G] .-
            this_scenario[:generators].Var_OM_cost_per_MWh[thermal_G]
      ) ./ this_scenario[:generators].Heat_rate_MMBTU_per_MWh[thermal_G]
    derived_Cost_per_MMBtu *= gas_mult * 5
    this_scenario[:generators].Var_Cost[thermal_G] .= 
                this_scenario[:generators].Var_OM_cost_per_MWh[thermal_G] .+
                derived_Cost_per_MMBtu .* this_scenario[:generators].Heat_rate_MMBTU_per_MWh[thermal_G]
    this_scenario[:generators].Start_Cost[thermal_G] .= 
                this_scenario[:generators].Start_cost_per_MW[thermal_G] .+
                derived_Cost_per_MMBtu .* this_scenario[:generators].Start_fuel_MMBTU_per_MW[thermal_G]
   
    push!(SCENARIOS, this_scenario)

end


# Create three variants (stochastic scenarios)
inputs_scenarios = [
    SCENARIOS[1]
    SCENARIOS[3]
    SCENARIOS[6]
]

# If want to give arbitrary weights and re-normalize
weights = [0, 0, 1]
weights = weights/sum(weights)

println("Scenario Weights: $(weights)")


# This is a deep truth about computers -- they represent numbers as so-called
# floating point numbers. How do you take the infinite real numbers to
# manageable byte length?
# A classic example is typing (0.1 + 0.2) == 0.3 in almost any programming
# language.
@assert isapprox(sum(weights), 1) "Sum of weights equals 1"
@assert length(weights) == length(inputs_scenarios)

# Build and solve model
stochastic_Model = generate_stochastic_model(base_inputs, inputs_scenarios, weights)

In [ ]:
function stochastic_uc(gen_df, loads, gen_var, network, mip_gap)
    UC = Model(HiGHS.Optimizer)
    set_optimizer_attribute(UC, "mip_rel_gap", mip_gap) 

    # isolate storage
    storage_df = gen_df[gen_df.stor .== 1, :]
    gen_df = gen_df[gen_df.stor .== 0, :]

    # gen subsets
    G_thermal = gen_df[gen_df[!,:up_time] .> 0,:r_id] # does NOT correspond to therm in gen_df
    G_nonthermal = gen_df[gen_df[!,:up_time] .== 0,:r_id]       
    G_var = gen_df[gen_df[!,:vre] .== 1, :r_id]
    G_nonvar = gen_df[gen_df[!,:vre] .== 0,:r_id]
    G_nt_nonvar = intersect(G_nonvar, G_nonthermal)
    B = storage_df.r_id # batteries
    G = gen_df.r_id
    L = network.network_lines
    T = unique(loads.hour)
    T_red = T[1:end-1]  # reduced time periods without last one
    Z = unique(gen_df.zone)

    # Decision variables   
    @variables(UC, begin
        GEN[G, T]  >= 0     # generation
        GENAUX[G_thermal, T] >= 0 # auxiliary generation variable
        COMMIT[G_thermal, T], Bin # commitment status (Bin=binary)
        START[G_thermal, T], Bin  # startup decision
        SHUT[G_thermal, T], Bin   # shutdown decision
        RESUP[G_thermal, T]       # up reserve capacity
        RESDN[G_thermal, T]       # down reserve capacity
        FLOW[l in L, t in T] >= 0 # power flows

        # battery vars
        0 <= SOC[b in B, T] <= storage_df[storage_df.r_id .== b, :existing_cap_mw][1]  # state of charge
        0 <= CHARGE[b in B, T] <= storage_df[storage_df.r_id .== b, :ramp_up_percentage][1] * 
                                    storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
        0 <= DISCHARGE[b in B, T] <= storage_df[storage_df.r_id .== b, :ramp_dn_percentage][1] * 
                                    storage_df[storage_df.r_id .== b, :existing_cap_mw][1]  
    end)
                
    # Objective function
    @objective(UC, Min, 
        sum( (gen_df[gen_df.r_id .== i,:heat_rate_mmbtu_per_mwh][1] * gen_df[gen_df.r_id .== i,:fuel_cost][1] +
            gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1]) * GEN[i,t] 
                        for i in G_nonvar for t in T) + 
        sum(gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1] * GEN[i,t] 
                        for i in G_var for t in T) + 
        sum(gen_df[gen_df.r_id .== i,:start_cost_per_mw][1] * 
            gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
            START[i,t] 
                        for i in G_thermal for t in T)
    )
    
    # OPF constraints

    # Auxiliary variable: total generation per zone and time
    @variable(UC, ZONE_GEN[z in Z, t in T] >= 0)

    @constraint(UC, ZoneGenDef[z in Z, t in T],
        ZONE_GEN[z, t] == sum(GEN[i, t] for i in G if gen_df[gen_df.r_id .== i, :zone][1] == z)
    )

    # power line capacities
    @constraint(UC, FlowMax[l in L, t in T],
        FLOW[l, t] <= network.line_max_flow_mw[l]
    )

    @constraint(UC, FlowMin[l in L, t in T],
        FLOW[l, t] >= -network.line_max_flow_mw[l]
    )

    # demand balance by zone
    @constraint(UC, NodalBalance[z in Z, t in T],
        ZONE_GEN[z, t] + 
        sum(DISCHARGE[b,t] - CHARGE[b,t] for b in B if storage_df[storage_df.r_id .== b, :zone][1] == z) +
        sum(FLOW[l, t] * network[l, "$z"] * (1 - network.line_loss[l]/2) for l in L if network[l, "$z"] != 0) ==
        loads[(loads.hour .== t) .& (loads.zone .== z), :demand][1]
    )

    # Capacity constraints (thermal generators requiring commitment)
    @constraint(UC, Cap_thermal_min[i in G_thermal, t in T], 
        GEN[i,t] >= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                        gen_df[gen_df.r_id .== i,:min_power][1])
    @constraint(UC, Cap_thermal_max[i in G_thermal, t in T], 
        GEN[i,t] <= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1])

    # Capacity constraints (non-variable generation not requiring commitment)
    @constraint(UC, Cap_nt_nonvar[i in G_nt_nonvar, t in T], 
        GEN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1])

    # Capacity constraints (variable generation)

    @constraint(UC, Cap_var[i in G_var, t in T], 
        GEN[i,t] <= gen_var[(gen_var.hour .== t) .& (gen_var.r_id .== i), :cf][1] *
                    gen_df[gen_df.r_id .== i, :existing_cap_mw][1])
    
    # battery constraints 
    @constraint(UC, cStateOfChargeEnd[b in B],
        SOC[b, T[end]] == 0.5 * storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
    )
    @constraint(UC, cStateOfChargeStart[b in B],
        SOC[b, T[1]] == 0.5 * storage_df[storage_df.r_id .== b, :existing_cap_mw][1] + # start charge
                (CHARGE[b, T[1]] * storage_df[storage_df.r_id .== b, :charge_efficiency][1] -
                DISCHARGE[b, T[1]] / storage_df[storage_df.r_id .== b, :discharge_efficiency][1])
    )
    @constraint(UC, cStateOfCharge[b in B, t in T[2:end]],
        SOC[b, t] == SOC[b, t-1] +
                    (CHARGE[b, t] * storage_df[storage_df.r_id .== b, :charge_efficiency][1] -
                    DISCHARGE[b, t] / storage_df[storage_df.r_id .== b, :discharge_efficiency][1])
    )

    # Unit commitment constraints
    @constraint(UC, Startup[i in G_thermal, t in T],
        COMMIT[i, t] >= sum(START[i, tt] 
                        for tt in intersect(T,
                            (t-gen_df[gen_df.r_id .== i,:up_time][1]):t)))

    @constraint(UC, Shutdown[i in G_thermal, t in T],
        1-COMMIT[i, t] >= sum(SHUT[i, tt] 
                        for tt in intersect(T,
                            (t-gen_df[gen_df.r_id .== i,:down_time][1]):t)))
    
    @constraint(UC, CommitmentStatus[i in G_thermal, t in T_red],
        COMMIT[i, t+1] - COMMIT[i, t] == START[i, t+1] - SHUT[i, t+1]
    )
    # New auxiliary variable GENAUX for generation above the minimum output level
    # for committed thermal units (only created for thermal generators)
    @constraint(UC, AuxGen[i in G_thermal, t in T],
        GENAUX[i,t] == GEN[i,t] - COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                        gen_df[gen_df.r_id .== i,:min_power][1])
    
    # Ramp equations for thermal generators (constraining GENAUX)
    @constraint(UC, RampUp_thermal[i in G_thermal, t in T_red], 
        GENAUX[i,t+1] - GENAUX[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_up_percentage][1] )

    @constraint(UC, RampDn_thermal[i in G_thermal, t in T_red], 
        GENAUX[i,t] - GENAUX[i,t+1] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1] )

    # Ramp equations for non-thermal generators (constraining total generation GEN)
    @constraint(UC, RampUp_nonthermal[i in G_nonthermal, t in T_red], 
        GEN[i,t+1] - GEN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_up_percentage][1] )

    @constraint(UC, RampDn[i in G, t in T_red], 
        GEN[i,t] - GEN[i,t+1] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1] )
    
                                 
    # Reserve equations
    # (1) Reserves limited by committed capacity of generator
    @constraint(UC, ResUpCap[i in G_thermal, t in T],
        RESUP[i,t] <= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1]
                        - GEN[i,t])

    @constraint(UC, ResDnCap[i in G_thermal, t in T],
        RESDN[i,t] <= GEN[i,t] - 
                        COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                        gen_df[gen_df.r_id .== i,:min_power][1])

    # (2) Reserves limited by ramp rates
    @constraint(UC, ResUpRamp[i in G_thermal, t in T],
        RESUP[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                          gen_df[gen_df.r_id .== i,:ramp_up_percentage][1])

    @constraint(UC, ResDnRamp[i in G_thermal, t in T],
        RESDN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                          gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1])

    # Compute reserve requirements
    ResReqUp = Dict(T[i] => 300  
              + 0.05 * loads[loads.hour .== T[i],:demand][1]
                    for i in 1:length(T))
    ResReqDn = Dict(T[i] => 0
              + 0.05 * loads[loads.hour .== T[i],:demand][1]                    
                    for i in 1:length(T))
                        
    # (3) Overall reserve requirements
    @constraint(UC, ResUpRequirement[t in T],
        sum(RESUP[i,t] for i in G_thermal) >= 
                ResReqUp[t])

    @constraint(UC, ResDnRequirement[t in T],
        sum(RESDN[i,t] for i in G_thermal) >= 
                ResReqDn[t])
    
    # Solve statement (! indicates runs in place)
    optimize!(UC)

    # Generation solution
    gen = value_to_df_2dim(value.(GEN))

    # Commitment status solution
    commit = value_to_df_2dim(value.(COMMIT))

    # Calculate curtailment
    curtail = DataFrame(r_id = Int[], hour = Int[], curt = Float64[])
    for i in G_var
        for t in T
            cf = gen_var[(gen_var.hour .== t) .& (gen_var.r_id .== i), :cf][1]
            existing_cap = gen_df[gen_df.r_id .== i, :existing_cap_mw][1]
            generation = value(GEN[i,t])
            curt_val = cf * existing_cap - generation
            push!(curtail, (r_id = i, hour = t, curt = curt_val))
        end
    end

    # Calculate total CO2 emissions
    total_co2 = sum(gen_df[gen_df.r_id .== i,:heat_rate_mmbtu_per_mwh][1] * 
                    gen_df[gen_df.r_id .== i,:co2_content_tons_per_mmbtu][1] * 
                    value(GEN[i,t]) for i in G_nonvar for t in T) +
                sum(gen_df[gen_df.r_id .== i,:start_fuel_mmbtu_per_mw][1] * 
                    gen_df[gen_df.r_id .== i,:co2_content_tons_per_mmbtu][1] * 
                    gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                    value(START[i,t]) for i in G_thermal for t in T)

    # battery vars
    charge_vals = Dict(b => [value(CHARGE[b, t]) for t in T] for b in B)
    discharge_vals = Dict(b => [value(DISCHARGE[b, t]) for t in T] for b in B)

    charge_df = DataFrame(
        r_id = repeat(B, inner=length(T)),
        hour = repeat(collect(T), outer=length(B)),
        value = vcat([charge_vals[b] for b in B]...)
    )

    discharge_df = DataFrame(
        r_id = repeat(B, inner=length(T)),
        hour = repeat(collect(T), outer=length(B)),
        value = vcat([discharge_vals[b] for b in B]...)
    )
    # Extract flow values into a DataFrame
    flow_vals = Dict((l, t) => value(FLOW[l, t]) for l in L, t in T)

    flows = DataFrame(
        line = repeat(L, inner=length(T)),
        hour = repeat(collect(T), outer=length(L)),
        flow = [flow_vals[(l, t)] for l in L for t in T]
    )

    # Store decision variables in a dictionary by time step
    decision_vars = Dict{Int, Dict{String, Any}}()
    
    for t in T
        decision_vars[t] = Dict{String, Any}(
            "GEN" => Dict(g => value(GEN[g, t]) for g in G),
            "GENAUX" => Dict(g => value(GENAUX[g, t]) for g in G_thermal),
            "COMMIT" => Dict(g => value(COMMIT[g, t]) for g in G_thermal),
            "RESUP" => Dict(g => value(RESUP[g, t]) for g in G_thermal),
            "RESDN" => Dict(g => value(RESDN[g, t]) for g in G_thermal),
            "SOC" => Dict(b => value(SOC[b, t]) for b in B)
        )
    end

    # Return the solution and objective as named tuple
    return (
        decision_vars,
        gen,
        commit,
        curtail,
        total_co2,
        charge_df,
        discharge_df,
        flows,
        reserves = value_to_df_2dim(value.(RESUP)),
        cost = objective_value(UC),
        status = termination_status(UC)
    )

end

In [ ]:
   demand_s        = [scenario_inputs[s][:demand]         for s in Sc] # Matrix T x Z
    variability_s   = [scenario_inputs[s][:variability]    for s in Sc] # Matrix T x G
    sample_weight_s = [scenario_inputs[s][:sample_weight]  for s in Sc] # Vector length T

In [ ]:
function robust_weighted_unit_commitment(scenarios, gen_df, network, mip_gap;
                                         penalty_demand=10000.0, penalty_curtail=50.0)
    UC = Model(HiGHS.Optimizer)
    set_optimizer_attribute(UC, "mip_rel_gap", mip_gap)
    
    # Isolate storage
    storage_df = gen_df[gen_df.stor .== 1, :]
    gen_df = gen_df[gen_df.stor .== 0, :]
    
    # Generator subsets
    G_thermal = gen_df[gen_df[!,:up_time] .> 0,:r_id]
    G_nonthermal = gen_df[gen_df[!,:up_time] .== 0,:r_id]
    G_var = gen_df[gen_df[!,:vre] .== 1, :r_id]
    G_nonvar = gen_df[gen_df[!,:vre] .== 0,:r_id]
    G_nt_nonvar = intersect(G_nonvar, G_nonthermal)
    B = storage_df.r_id
    G = gen_df.r_id
    L = network.network_lines
    
    # Time periods (same across all scenarios)
    T = unique(scenarios[1].loads.hour)
    T_red = T[1:end-1]
    Z = unique(gen_df.zone)
    
    # Number of scenarios
    S = 1:length(scenarios)
    
    # ==========================================
    # DECISION VARIABLES - SINGLE SET (not scenario-dependent)
    # ==========================================
    @variables(UC, begin
        GEN[G, T] >= 0                  # Generation (same for all scenarios)
        GENAUX[G_thermal, T] >= 0       # Auxiliary generation
        COMMIT[G_thermal, T], Bin       # Commitment decisions
        START[G_thermal, T], Bin        # Startup decisions
        SHUT[G_thermal, T], Bin         # Shutdown decisions
        RESUP[G_thermal, T] >= 0        # Up reserves
        RESDN[G_thermal, T] >= 0        # Down reserves
        FLOW[l in L, t in T]            # Power flows (same for all scenarios)
        
        # Battery vars (same for all scenarios)
        0 <= SOC[b in B, T] <= storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
        0 <= CHARGE[b in B, T] <= storage_df[storage_df.r_id .== b, :ramp_up_percentage][1] * 
                                    storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
        0 <= DISCHARGE[b in B, T] <= storage_df[storage_df.r_id .== b, :ramp_dn_percentage][1] * 
                                    storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
    end)
    
    # ==========================================
    # SLACK VARIABLES - Track violations per scenario
    # ==========================================
    @variables(UC, begin
        UNMET_DEMAND[s in S, z in Z, t in T] >= 0   # Demand not met in scenario s
        CURTAIL[s in S, i in G_var, t in T] >= 0    # VRE curtailment in scenario s
        EXCESS_GEN[s in S, z in Z, t in T] >= 0     # Excess generation in scenario s
    end)
    
    # ==========================================
    # OBJECTIVE: Expected cost + Expected penalties
    # ==========================================
    @objective(UC, Min,
        # Fixed operational costs (same across scenarios)
        sum(gen_df[gen_df.r_id .== i,:start_cost_per_mw][1] * 
            gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
            START[i,t] for i in G_thermal for t in T) +
        
        # Generation costs (same across scenarios)
        sum((gen_df[gen_df.r_id .== i,:heat_rate_mmbtu_per_mwh][1] * 
             gen_df[gen_df.r_id .== i,:fuel_cost][1] +
             gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1]) * GEN[i,t] 
            for i in G_nonvar for t in T) +
        sum(gen_df[gen_df.r_id .== i,:var_om_cost_per_mwh][1] * GEN[i,t] 
            for i in G_var for t in T) +
        
        # Expected penalty for violations across scenarios
        sum(scenarios[s].probability * (
            # Penalty for unmet demand (VERY HIGH)
            sum(penalty_demand * UNMET_DEMAND[s,z,t] for z in Z for t in T) +
            # Penalty for curtailment (moderate)
            sum(penalty_curtail * CURTAIL[s,i,t] for i in G_var for t in T) +
            # Small penalty for excess generation
            sum(10.0 * EXCESS_GEN[s,z,t] for z in Z for t in T)
        ) for s in S)
    )
    
    # ==========================================
    # CONSTRAINTS PER SCENARIO
    # Track how well our single decision works in each scenario
    # ==========================================
    for s in S
        scenario = scenarios[s]
        
        # Auxiliary variable: total generation per zone
        @variable(UC, ZONE_GEN[z in Z, t in T] >= 0)
        @constraint(UC, [z in Z, t in T],
            ZONE_GEN[z, t] == sum(GEN[i, t] for i in G if gen_df[gen_df.r_id .== i, :zone][1] == z)
        )
        
        # RELAXED demand balance - allow unmet demand and excess generation
        @constraint(UC, [z in Z, t in T],
            ZONE_GEN[z, t] + 
            sum(DISCHARGE[b,t] - CHARGE[b,t] for b in B if storage_df[storage_df.r_id .== b, :zone][1] == z) +
            sum(FLOW[l,t] * network[l, "$z"] * (1 - network.line_loss[l]/2) for l in L if network[l, "$z"] != 0) +
            UNMET_DEMAND[s,z,t] - EXCESS_GEN[s,z,t] ==
            scenario.loads[(scenario.loads.hour .== t) .& (scenario.loads.zone .== z), :demand][1]
        )
        
        # VRE generation is limited by scenario-specific capacity factor
        # Allow curtailment (generation below available)
        @constraint(UC, [i in G_var, t in T],
            GEN[i,t] + CURTAIL[s,i,t] == 
                scenario.gen_var[(scenario.gen_var.hour .== t) .& (scenario.gen_var.r_id .== i), :cf][1] *
                gen_df[gen_df.r_id .== i, :existing_cap_mw][1]
        )
    end
    
    # ==========================================
    # DETERMINISTIC CONSTRAINTS (not scenario-dependent)
    # ==========================================
    
    # Power line capacities
    @constraint(UC, [l in L, t in T],
        FLOW[l,t] <= network.line_max_flow_mw[l]
    )
    @constraint(UC, [l in L, t in T],
        FLOW[l,t] >= -network.line_max_flow_mw[l]
    )
    
    # Capacity constraints (thermal)
    @constraint(UC, [i in G_thermal, t in T],
        GEN[i,t] >= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                    gen_df[gen_df.r_id .== i,:min_power][1]
    )
    @constraint(UC, [i in G_thermal, t in T],
        GEN[i,t] <= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1]
    )
    
    # Capacity constraints (non-thermal, non-variable)
    @constraint(UC, [i in G_nt_nonvar, t in T],
        GEN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1]
    )
    
    # VRE capacity - use AVERAGE across scenarios (or min/max if you prefer)
    avg_cf = Dict()
    for i in G_var
        for t in T
            cfs = [scenarios[s].gen_var[(scenarios[s].gen_var.hour .== t) .& 
                                        (scenarios[s].gen_var.r_id .== i), :cf][1] for s in S]
            avg_cf[(i,t)] = sum(scenarios[s].probability * cfs[s] for s in S)
        end
    end
    
    @constraint(UC, [i in G_var, t in T],
        GEN[i,t] <= avg_cf[(i,t)] * gen_df[gen_df.r_id .== i, :existing_cap_mw][1]
    )
    
    # Battery constraints
    @constraint(UC, [b in B],
        SOC[b, T[end]] == 0.5 * storage_df[storage_df.r_id .== b, :existing_cap_mw][1]
    )
    @constraint(UC, [b in B],
        SOC[b, T[1]] == 0.5 * storage_df[storage_df.r_id .== b, :existing_cap_mw][1] +
                (CHARGE[b, T[1]] * storage_df[storage_df.r_id .== b, :charge_efficiency][1] -
                DISCHARGE[b, T[1]] / storage_df[storage_df.r_id .== b, :discharge_efficiency][1])
    )
    @constraint(UC, [b in B, t in T[2:end]],
        SOC[b, t] == SOC[b, t-1] +
                (CHARGE[b, t] * storage_df[storage_df.r_id .== b, :charge_efficiency][1] -
                DISCHARGE[b, t] / storage_df[storage_df.r_id .== b, :discharge_efficiency][1])
    )
    
    # Unit commitment constraints
    @constraint(UC, [i in G_thermal, t in T],
        COMMIT[i, t] >= sum(START[i, tt] for tt in intersect(T, (t-gen_df[gen_df.r_id .== i,:up_time][1]):t))
    )
    @constraint(UC, [i in G_thermal, t in T],
        1-COMMIT[i, t] >= sum(SHUT[i, tt] for tt in intersect(T, (t-gen_df[gen_df.r_id .== i,:down_time][1]):t))
    )
    @constraint(UC, [i in G_thermal, t in T_red],
        COMMIT[i, t+1] - COMMIT[i, t] == START[i, t+1] - SHUT[i, t+1]
    )
    
    # Auxiliary generation variable
    @constraint(UC, [i in G_thermal, t in T],
        GENAUX[i,t] == GEN[i,t] - COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                        gen_df[gen_df.r_id .== i,:min_power][1]
    )
    
    # Ramp constraints
    @constraint(UC, [i in G_thermal, t in T_red],
        GENAUX[i,t+1] - GENAUX[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_up_percentage][1]
    )
    @constraint(UC, [i in G_thermal, t in T_red],
        GENAUX[i,t] - GENAUX[i,t+1] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1]
    )
    @constraint(UC, [i in G_nonthermal, t in T_red],
        GEN[i,t+1] - GEN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_up_percentage][1]
    )
    @constraint(UC, [i in G, t in T_red],
        GEN[i,t] - GEN[i,t+1] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                                 gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1]
    )
    
    # Reserve constraints
    @constraint(UC, [i in G_thermal, t in T],
        RESUP[i,t] <= COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] - GEN[i,t]
    )
    @constraint(UC, [i in G_thermal, t in T],
        RESDN[i,t] <= GEN[i,t] - COMMIT[i, t] * gen_df[gen_df.r_id .== i,:existing_cap_mw][1] *
                      gen_df[gen_df.r_id .== i,:min_power][1]
    )
    @constraint(UC, [i in G_thermal, t in T],
        RESUP[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                      gen_df[gen_df.r_id .== i,:ramp_up_percentage][1]
    )
    @constraint(UC, [i in G_thermal, t in T],
        RESDN[i,t] <= gen_df[gen_df.r_id .== i,:existing_cap_mw][1] * 
                      gen_df[gen_df.r_id .== i,:ramp_dn_percentage][1]
    )
    
    # Reserve requirements - use average demand across scenarios
    avg_demand = Dict()
    for t in T
        demands = [scenarios[s].loads[scenarios[s].loads.hour .== t, :demand][1] for s in S]
        avg_demand[t] = sum(scenarios[s].probability * demands[s] for s in S)
    end
    
    @constraint(UC, [t in T],
        sum(RESUP[i,t] for i in G_thermal) >= 300 + 0.05 * avg_demand[t]
    )
    @constraint(UC, [t in T],
        sum(RESDN[i,t] for i in G_thermal) >= 0 + 0.05 * avg_demand[t]
    )
    
    # Solve
    optimize!(UC)
    
    # ==========================================
    # EXTRACT SOLUTION
    # ==========================================
    
    # Single generation decision
    gen = DataFrame(
        r_id = repeat(G, inner=length(T)),
        hour = repeat(collect(T), outer=length(G)),
        gen = [value(GEN[g, t]) for g in G for t in T]
    )
    
    # Commitment decisions
    commit = DataFrame(
        r_id = repeat(G_thermal, inner=length(T)),
        hour = repeat(collect(T), outer=length(G_thermal)),
        commit = [value(COMMIT[g, t]) for g in G_thermal for t in T]
    )
    
    # Violations per scenario
    scenario_violations = []
    for s in S
        unmet = sum(value(UNMET_DEMAND[s,z,t]) for z in Z for t in T)
        curtail = sum(value(CURTAIL[s,i,t]) for i in G_var for t in T)
        excess = sum(value(EXCESS_GEN[s,z,t]) for z in Z for t in T)
        
        push!(scenario_violations, (
            scenario = s,
            probability = scenarios[s].probability,
            unmet_demand = unmet,
            curtailment = curtail,
            excess_gen = excess
        ))
    end
    
    # Battery solution
    charge_df = DataFrame(
        r_id = repeat(B, inner=length(T)),
        hour = repeat(collect(T), outer=length(B)),
        value = [value(CHARGE[b, t]) for b in B for t in T]
    )
    
    discharge_df = DataFrame(
        r_id = repeat(B, inner=length(T)),
        hour = repeat(collect(T), outer=length(B)),
        value = [value(DISCHARGE[b, t]) for b in B for t in T]
    )
    
    # Flow solution
    flows = DataFrame(
        line = repeat(L, inner=length(T)),
        hour = repeat(collect(T), outer=length(L)),
        flow = [value(FLOW[l, t]) for l in L for t in T]
    )
    
    return (
        gen = gen,
        commit = commit,
        charge = charge_df,
        discharge = discharge_df,
        flows = flows,
        violations = scenario_violations,
        reserves = DataFrame(
            r_id = repeat(G_thermal, inner=length(T)),
            hour = repeat(collect(T), outer=length(G_thermal)),
            resup = [value(RESUP[g, t]) for g in G_thermal for t in T]
        ),
        cost = objective_value(UC),
        status = termination_status(UC)
    )
end

# ==========================================
# EXAMPLE USAGE
# ==========================================

# Generate scenarios
scenarios = generate_scenarios_perturbation(gen_var_multi, loads_multi, 10,
                                           wind_std=0.15, solar_std=0.15, load_std=0.05)

# Solve with robust weighted approach
solution = robust_weighted_unit_commitment(scenarios, gen_df, network, 0.01,
                                          penalty_demand=10000.0,   # Very high - must meet demand
                                          penalty_curtail=50.0)      # Moderate - ok to curtail some

println("Total cost: ", solution.cost)
println("\nViolations by scenario:")
for v in solution.violations
    println("Scenario $(v.scenario) (prob=$(v.probability)):")
    println("  Unmet demand: $(v.unmet_demand) MWh")
    println("  Curtailment: $(v.curtailment) MWh")
    println("  Excess gen: $(v.excess_gen) MWh")
end